In [3]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


from nltk import tokenize
#nltk.download()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/andrin/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
art_df_full = pd.read_csv("US_BANKS_ALL.csv")
art_df_full["row_id"] = art_df_full.index + 1

bankname_df = art_df_full.copy()
bankname_df = bankname_df.fillna('')
bankname_df['TEXT'] = bankname_df['TEXT'].str.casefold()


In [29]:
bank_dict = {"HSBC": ["hsbc"], 
            "BARCLAYS": ["barclays","barclay"],
            "STCHARTERED": ["standard chartered","standard","chartered"],
            "Royal Bank of Scotland": ["royal bank of scotland", "rbs", "royal"],
            "Bank of Scotland": ["bank of scotland", " bos "],
            "National Westminster": ["national westminster"," nw ","westminster"],
            "Lloyds Bank": ["lloyds bank","lloyds","lloyd"],
            "Midland Bank": ["midland bank","midland"],
            "Rabobank": ["rabobank","rabo"],
            "Barclays de Zoete Wedd": ["barclays","zoete","wedd"],
            }

In [6]:
bank_dict = {"Bank of America": ["bank of america","boa"], 
            "Comerica Inc": ["comerica"],
            "Goldman Sachs": ["goldman sachs","goldman","sachs"],
            "Wells Fargo": ["wells fargo", "wells", "fargo"]
            }




In [7]:

for i in range(len(bankname_df['BANK'])):

    text = bankname_df['TEXT'][i]

    phrases = tokenizer.tokenize(text)

    bank_phrases = []

    potential_names = bank_dict[bankname_df['BANK'][i]]

    for phrase in phrases:

        for name in potential_names:
            if name in phrase:
                bank_phrases.append(phrase)

    new_text = " ".join(bank_phrases)

    bankname_df.at[i,'TEXT'] = new_text


In [8]:
art_df = bankname_df[['row_id', 'TEXT']].copy()

art_df['TEXT'] = art_df['TEXT'].str.replace("[^a-zA-Z#]", " ")

art_df['TEXT'] = art_df['TEXT'].str.casefold()

art_df = art_df.fillna('')

In [9]:
results_df = pd.DataFrame()
results_df['row_id'] = ['99999999999']
results_df['sentiment_type'] = pd.NA
results_df['sentiment_score'] = 0

In [10]:
sid = SentimentIntensityAnalyzer()
t_df = results_df

for index, row in art_df.iterrows():

    #print(index)

    scores = sid.polarity_scores(row[1])

    for key, value in scores.items():

        temp = [key,value,row[0]]

        results_df['row_id'] = row[0]
        results_df['sentiment_type'] = key
        results_df['sentiment_score'] = value

        t_df = t_df.append(results_df)

t_df_cleaned = t_df[t_df.row_id != '99999999999']

t_df_cleaned = t_df_cleaned.drop_duplicates()

t_df_cleaned = t_df[t_df.sentiment_type == 'compound']

In [11]:
article_scores_df = pd.merge(art_df_full, t_df_cleaned, on='row_id', how='inner')

article_scores_df['YEAR'] = article_scores_df['DATE'].str.slice(0,4)

article_scores_df = article_scores_df.drop(article_scores_df.columns[[7]] , axis=1)

bank_phrase_scores = article_scores_df[['row_id', 'sentiment_score']].copy()

bank_phrase_scores.columns = ['row_id','phrase_score']

bank_phrase_scores.to_csv("bank_phrase_scores.csv", encoding='utf-8', index=False)
